In [ ]:
import yfinance as yf
import datetime
import pandas as pd
from pandas_datareader import data as web
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tick = input()

In [ ]:
y_ds = yf.download(tickers=tick,start='2013-01-02', end='2018-12-28')

In [ ]:
y_test = pd.Series(yf.download(tickers=tick,start='2019-01-02', end='2019-06-01',)['Adj Close'])

In [ ]:
trace1 = {
    'x': y_ds.index,
    'open': y_ds.Open,
    'close': y_ds.Close,
    'high': y_ds.High,
    'low': y_ds.Low,
    'type': 'candlestick',
    'name': tick,
    'showlegend': True
}

In [ ]:
data = [trace1]
layout = go.Layout({
    'title': {
        'text': tick,
        'font': {
            'size': 15
        }
    }
})

In [ ]:
fig = go.Figure()
fig.add_trace(trace1)
fig.update_layout(layout)
fig.show()

The data clearly shows that there is no seasonality, but there may be a relative trend. 

In [ ]:
y_fit = pd.Series(y_ds['Adj Close'])
figadj = px.line(y_fit)
figadj.show()

## Single Exponential Smoothing

Here we will create three models in which we will consider three different 𝜶 values.

In [ ]:
y_fit.index = pd.DatetimeIndex(y_fit.index).to_period('M')

𝜶 = 0.2

In [ ]:
ins1 = SimpleExpSmoothing(y_fit, initialization_method="heuristic").fit(smoothing_level=0.2,optimized=False)
ins_cast1 = ins1.forecast(6).rename(r'$\alpha=0.2$')

𝜶 = 0.8

In [ ]:
ins2 = SimpleExpSmoothing(y_fit, initialization_method="heuristic").fit(smoothing_level=0.8,optimized=False)
ins_cast2 = ins2.forecast(6).rename(r'$\alpha=0.8$')

𝜶 value automatically optimized by statsmodel

In [ ]:
ins3 = SimpleExpSmoothing(y_fit, initialization_method="estimated").fit()
ins_cast3 = ins3.forecast(6).rename(r'$\alpha=%s$'%ins3.model.params['smoothing_level'])

In [ ]:
ins_cast1.index = ins_cast1.index.to_timestamp()
ins_cast1 *= (y_fit[-1]/ins_cast1[0])
ins_cast2.index = ins_cast2.index.to_timestamp()
ins_cast2 *= (y_fit[-1]/ins_cast2[0])
ins_cast3.index = ins_cast3.index.to_timestamp()
ins_cast3 *= (y_fit[-1]/ins_cast3[0])

In [ ]:
plt.figure(figsize=(20, 8))
plt.grid(True)
plt.title("Simple Exponential Smoothing for "+tick)
plt.plot(ins_cast1, color='green', label=r'$\alpha=%s$0.2')
plt.plot(ins_cast2, color='red', label=r'$\alpha=%s$0.8')
plt.plot(ins_cast3, color='blue', label = r'$\alpha=%s$'%ins3.model.params['smoothing_level'])
plt.legend()

In [ ]:
plt.figure(figsize=(20, 8))
plt.grid(True)
plt.title("Simple Exponential Smoothing for "+tick)
plt.plot(y_test)
plt.plot(ins_cast1, color='green', label=r'$\alpha=%s$0.2')
plt.plot(ins_cast2, color='red', label=r'$\alpha=%s$0.8')
plt.plot(ins_cast3, color='blue', label = r'$\alpha=%s$'%ins3.model.params['smoothing_level'])
plt.legend()

## Double Exponential Smoothing

In [ ]:
Holt_fit1 = Holt(y_fit, initialization_method="estimated").fit(smoothing_level=0.8, smoothing_trend=0.2, optimized=False)
Holt_fcast1 = Holt_fit1.forecast(6).rename("Holt's linear trend")
Holt_fit2 = Holt(y_fit, exponential=True, initialization_method="estimated").fit(smoothing_level=0.8, smoothing_trend=0.2, optimized=False)
Holt_fcast2 = Holt_fit2.forecast(6).rename("Exponential trend")
Holt_fit3 = Holt(y_fit, damped_trend=True, initialization_method="estimated").fit(smoothing_level=0.8, smoothing_trend=0.2)
Holt_fcast3 = Holt_fit3.forecast(6).rename("Additive damped trend")

In [ ]:
Holt_fcast1.index = Holt_fcast1.index.to_timestamp()
Holt_fcast1 *= (y_fit[-1]/Holt_fcast1[0])
Holt_fcast2.index = Holt_fcast2.index.to_timestamp()
Holt_fcast2 *= (y_fit[-1]/Holt_fcast2[0])
Holt_fcast3.index = Holt_fcast3.index.to_timestamp()
Holt_fcast3 *= (y_fit[-1]/Holt_fcast3[0])

In [ ]:
plt.figure(figsize=(20, 8))
plt.grid(True)
plt.title("Double Exponential Smoothing for " + tick)
plt.plot(Holt_fcast1, color='green', label="Holt's linear trend")
plt.plot(Holt_fcast2, color='red', label="Exponential trend")
plt.plot(Holt_fcast3, color='blue', label ="Additive damped trend")
plt.legend()

In [ ]:
plt.figure(figsize=(20, 8))
plt.grid(True)
plt.title("Double Exponential Smoothing for " + tick)
plt.plot(y_test)
plt.plot(Holt_fcast1, color='green', label="Holt's linear trend")
plt.plot(Holt_fcast2, color='red', label="Exponential trend")
plt.plot(Holt_fcast3, color='blue', label ="Additive damped trend")
plt.legend()